In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import numpy as np
import random
import glob
import os

In [ ]:
paths = glob.glob(os.path.join('val2017','*'))
print(len(paths))

In [ ]:
train_paths = paths[:4500]
val_paths = paths[4500:]

In [ ]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

In [ ]:
def add_speckle(s1, looks = 4):
    # Numpy Gamma Distribution is defined in the shape-scale form
    # Mean 1 Var 1/looks
    gamma_shape = looks
    gamma_scale = 1/looks

    noise = np.random.gamma(gamma_shape, 
                            gamma_scale, 
                            s1.shape[0]*s1.shape[1]).reshape(s1.shape)
    s1 = s1*noise

    return s1, noise

def add_noise(s1, var = 0.2):
    row,col = s1.shape
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = s1 + gauss
    noisy = np.clip(noisy, 0.0, 1.0)
    
    return noisy, gauss


In [ ]:
def norm(img):
    return (img - img.min())/(img.max() - img.min())

In [ ]:
def dataL(paths, batch_size, shape, var = 1):
    
    batch_in  = np.zeros((batch_size, shape, shape, 1))
    batch_out = np.zeros((batch_size, shape, shape, 1))
    random.shuffle(paths)
    counter = 0
    
    while True:
        
        if counter>= len(paths) - batch_size:
            counter = 0
            random.shuffle(paths)
            
        for i in range(batch_size):
            img = plt.imread(paths[counter])[:shape, :shape, :]
            img = img/img.max()
            
            batch_out[i,:,:,0] = rgb2gray(img)
            batch_in[i,:,:,0], _  = add_noise(batch_out[i, :,:,0], var = var)#add_speckle(batch_out[i, :,:,0], looks = 4)
            #batch_in[i,:,:,0], _  = add_speckle(batch_out[i, :,:,0], looks = 4)
            
            #batch_in[i,:,:,0] = (batch_in[i,:,:,0] - batch_in[i,:,:,0].min()) / (batch_in[i,:,:,0].max() - batch_in[i,:,:,0].min())
            
            counter += 1
            
        yield  np.clip(batch_in, 0.0, 1.0), np.clip(batch_out, 0.0, 1.0)

# Train-Test model

In [ ]:
from Model import CNNSpeckleFilter
N_LAYER = 12
shape = 256
epochs = 60
batch_size = 12
varias = [0.01, 0.02, 0.05, 0.08, 0.1]#[1, 2, 4]

for var in varias:
    
    # Train
    speckle_filter = CNNSpeckleFilter(input_shape=(shape,shape,1), n_layers=N_LAYER)
    train_gen = dataL(train_paths, batch_size, shape, var=var)
    val_gen = dataL(val_paths, batch_size, shape, var=var)
    train_step = len(train_paths)//batch_size
    val_step = len(val_paths)//batch_size
    #history = speckle_filter.train_model(epochs, train_gen, val_gen, train_step, val_step)
    #speckle_filter.model.save_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_speckle-var_'+str(var)+'.h5')
    #speckle_filter.model.load_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_speckle-var_'+str(var)+'.h5')
    speckle_filter.model.load_weights('weights/noSAR_model_'+str(N_LAYER)+'layers_ssim_mse_clipin_tanh_v3_gauss-var_'+str(var)+'.h5')
    
    ## Test

    test_paths = glob.glob(os.path.join('coco_test_dataset','*'))
    batch_size = len(test_paths)

    val_gen = dataL(test_paths, batch_size, shape, var = var)
    x,y = next(iter(val_gen))
    yp = speckle_filter.model.predict(x)

    for i in range(batch_size):

        fig, axes = plt.subplots(nrows=1,ncols=5, figsize = (30,10))
        axes[0].imshow(x[i,:,:,0], cmap = 'gray')
        axes[0].axis(False)

        axes[1].imshow(y[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[1].axis(False)

        n = (yp[i,:,:,0] - yp[i,:,:,0].min()) / (yp[i,:,:,0].max() - yp[i,:,:,0].min())
        axes[2].imshow(n, vmax = 1, cmap = 'gray')
        axes[2].axis(False)

        axes[3].imshow(yp[i,:,:,0], vmax = 1, cmap = 'gray')
        axes[3].axis(False)

        axes[4].imshow(np.abs(x[i,:,:,0] - yp[i,:,:,0]), vmax = 1, cmap = 'gray')
        axes[4].axis(False)
        plt.show()
        plt.close()

        plt.imsave('results_speckle\\b_{}_noise-l_{}.png'.format(i,var), x[i,:,:,0], cmap = 'gray')
        plt.imsave('results_speckle\\b_{}_gt-l_{}.png'.format(i,var), y[i,:,:,0], cmap = 'gray')
        plt.imsave('results_speckle\\b_{}_pred-l_{}.png'.format(i,var), n, cmap = 'gray')
        print('Looks {}'.format(var))
        print('PSNR(GT,N): ', peak_signal_noise_ratio(y[i,:,:,0], x[i,:,:,0]))
        print('PSNR(GT,P): ', peak_signal_noise_ratio(y[i,:,:,0], n))
              
        print('SSIM(GT,N): ', structural_similarity(y[i,:,:,0], x[i,:,:,0]))
        print('SSIM(GT,P): ', structural_similarity(y[i,:,:,0], n))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tests = ['0','1','2','3','4','5','6']
powers = ['0.01', '0.02', '0.05', '0.08', '0.1'] 

for test in tests:
    for power in powers:
        gt    = plt.imread('results_gaussian\\b_{}_gt-p_{}.png'.format(test, power))[:,:,0]
        noisy = plt.imread('results_gaussian\\b_{}_noise-p_{}.png'.format(test, power))[:,:,0]
        pred  = plt.imread('results_gaussian\\b_{}_pred-p_{}.png'.format(test, power))[:,:,0]
        
        print(' --------------------- TEST {} --------------------'.format(test))
        print(' ========== Power {}'.format(power))
              
        print('PSNR(GT,N): ', peak_signal_noise_ratio(gt, noisy))
        print('PSNR(GT,P): ', peak_signal_noise_ratio(gt, pred))
              
        print('SSIM(GT,N): ', structural_similarity(gt, noisy))
        print('SSIM(GT,P): ', structural_similarity(gt, pred))
        
        fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (10,5))
        axes[0].imshow(gt)
        axes[1].imshow(noisy)
        axes[2].imshow(pred)
        plt.show()
        plt.close()

In [ ]:
tests = ['0','1','2','3','4','5','6']
powers = ['1', '2', '4'] 

for test in tests:
    for power in powers:
        gt    = plt.imread('results_speckle\\b_{}_gt-l_{}.png'.format(test, power))[:,:,0]
        noisy = plt.imread('results_speckle\\b_{}_noise-l_{}.png'.format(test, power))[:,:,0]
        pred  = plt.imread('results_speckle\\b_{}_pred-l_{}.png'.format(test, power))[:,:,0]
        
        print(' --------------------- TEST {} --------------------'.format(test))
        print(' ========== L {}'.format(power))
              
        print('PSNR(GT,N): ', peak_signal_noise_ratio(gt, noisy))
        print('PSNR(GT,P): ', peak_signal_noise_ratio(gt, pred))
        print('SSIM(GT,N): ', structural_similarity(gt, noisy))   
        print('SSIM(GT,P): ', structural_similarity(gt, pred))
        
        fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (10,5))
        axes[0].imshow(gt)
        axes[1].imshow(noisy)
        axes[2].imshow(pred)
        plt.show()
        plt.close()
        